In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
from selenium_stealth import stealth
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.webdriver.common.alert import Alert

import undetected_chromedriver as uc
import pandas as pd
import numpy as np
import time
import csv
import selenium

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 최종 코드
max_reviews = 6000
star5 = []
star4 = []
star3 = []
star2 = []
star1 = []

print(f"===== 현재 URL 작업 중: {url} =====")
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled") # 크롤링 감지 시스템 우회
driver = webdriver.Chrome(options=options)
driver.maximize_window()

# 크롤링 감지 시스템 우회
stealth(driver,
        languages=["ko-KR", "ko"],
        vendor="Google Inc.",
        platform="Win64",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
)
url = "https://www.auction.co.kr/n/list?category=51190500&s=8"
driver.get(url)
time.sleep(3)

# 원하는 제품이 보일 때까지 스크롤 내리기
body = driver.find_element("tag name", "body")
body.send_keys(Keys.PAGE_DOWN)
time.sleep(0.5)

# 원하는 제품 선택
elements = driver.find_elements(By.CSS_SELECTOR, 'span.text--itemcard_title.ellipsis')
if len(elements) >= 4:
    elements[3].click() # 감귤 같은 경우에는 원하는 제품이 4번째에 있으므로
else:
    print("요소가 4개 미만입니다.")

time.sleep(3)

# 화면 새로고침
driver.switch_to.window(driver.window_handles[-1])
time.sleep(2)

# 상세설명 - 구매후기 - 상품문의 - 교환/반품 탭 보일 때까지 스크롤 내리기
body2 = driver.find_element("tag name", "body")
# body2 = driver.find_element(By.TAG_NAME, "body")
for _ in range(0, 2):
    body2.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)

time.sleep(2)

# 구매후기 탭 클릭
tap = driver.find_element(By.ID, "tap_moving_2")
tap.click()
time.sleep(3)

# 별점별 보기 탭 클릭
driver.find_element(By.CSS_SELECTOR, "li.list-item.list-item__score").click()
time.sleep(2)

# 1~5점 
scores = ['li[data-filter="score-1"] button.js-button', 'li[data-filter="score-2"] button.js-button', 
          'li[data-filter="score-3"] button.js-button', 'li[data-filter="score-4"] button.js-button', 
          'li[data-filter="score-5"] button.js-button']

for idx, score in enumerate(scores):
    # 크롤링 전에 점수별 최대 수 확인 -> 6,000개 넘거나 같으면 해당 점수의 리뷰는 스킵
    if idx == 0 and len(star1) >= max_reviews:
        continue
    elif idx == 1 and len(star2) >= max_reviews:
        continue
    elif idx == 2 and len(star3) >= max_reviews:
        continue
    elif idx == 3 and len(star4) >= max_reviews:
        continue
    elif idx == 4 and len(star5) >= max_reviews:
        continue

    # 1~5점 리뷰 버튼 클릭
    try:
        rating_button = driver.find_element(By.CSS_SELECTOR, score)
        rating_button.click()
    
    except:
        print("리뷰 버튼 클릭 실패:")
        break
        
    time.sleep(2)
    
    seen_count = 0

    while seen_count < max_reviews: # 페이지당 최대 40개의 리뷰가 있다.
        time.sleep(2)
        items = driver.find_elements(By.CSS_SELECTOR, "div.box__review-item")
    
        if not items:
            print("새 리뷰 없음")
            break
    
        for item in items:
            grade = idx + 1 # 평점
            try:
                # 해당 리뷰 내 더보기 버튼이 있으면 클릭
                more_button = item.find_element(By.CSS_SELECTOR, "button.button.button__view-more")
                driver.execute_script("arguments[0].click();", more_button)
                time.sleep(0.5)
                # print("더보기 버튼 클릭 완료.")
                
            except:
                pass
                # print("더보기 버튼이 없습니다.")

            # 리뷰
            try:
                full_review = item.find_element(By.CSS_SELECTOR, "div.box__review-text > p.text").text.strip()
                if full_review == "":
                    continue
                # print("리뷰 있음")
            except:
                continue
                # print("리뷰 없음")

            # 이미지 유무 and 이미지 개수
            try:
                image_buttons = item.find_elements(By.CSS_SELECTOR, "ul.list li.list-item a.link")
                image_count = len(image_buttons)
                has_image = "Y" if image_count > 0 else "N"
            except:
                image_count = 0
                has_image = "N"

            # 도움돼요 수
            try:
                help_count = item.find_element(By.CSS_SELECTOR, "div.box__helpful span.text__count").text
            except:
                help_count = "0"

            # 작성일자
            try:
                date = item.find_element(By.CSS_SELECTOR, "div.box__info p.text__date").text
            except:
                date = ""
    
            result = {
                "대분류": '과일',
                "소분류": '감귤',
                "작성일": date,
                "평점": grade,
                "리뷰": full_review.strip(),
                "이미지포함": has_image,
                "이미지수": image_count,
                "도움돼요수": int(help_count.replace(",", "")),
                "상품URL": url
            }

            # 데이터가 균일하게 들어있는지 쉽게 확인하기 위해 1~5점 데이터를 따로 저장
            if grade == 1:
                star1.append(result)
            elif grade == 2:
                star2.append(result)
            elif grade == 3:
                star3.append(result)
            elif grade == 4:
                star4.append(result)
            elif grade == 5:
                star5.append(result)

        seen_count += len(items)
        print(f"평점 {grade}의 {seen_count}개의 리뷰 저장 완료")

        # 다음 페이지를 넘기는 버튼인 '다음' 버튼이 보일 때까지 스크롤 내리기
        scroll_attempts = 0
        max_scrolls = 30
        more_button = None
    
        while scroll_attempts < max_scrolls:
            try:
                more_button = driver.find_element(By.CSS_SELECTOR, 
                                                  "div.box__list-review.box__list-review--active div.box__pagination a.link__page-move.link__page-next")
                if more_button.is_displayed():
                    break
            except:
                pass
    
            driver.execute_script("window.scrollBy(0, 500);")
            time.sleep(0.5)
            scroll_attempts += 1
    
        if more_button and more_button.is_displayed():
            more_button.click()
            print("다음 클릭 완료")
            time.sleep(2)
        else:
            print("다음 버튼이 보이지 않음.")
            body2.send_keys(Keys.HOME)
            for _ in range(3):
                body2.send_keys(Keys.PAGE_DOWN)
                time.sleep(0.5)
            break

driver.quit()
print(f"=== {url} 작업 완료 ===\n")

# 예시: star5 리뷰 출력
print(f"총 수집된 5점 리뷰 개수: {len(star5)}")
print(f"총 수집된 4점 리뷰 개수: {len(star4)}")
print(f"총 수집된 3점 리뷰 개수: {len(star3)}")
print(f"총 수집된 2점 리뷰 개수: {len(star2)}")
print(f"총 수집된 1점 리뷰 개수: {len(star1)}")

In [ ]:
# 각각 저장
pd.DataFrame(star5).to_csv("파일명.csv", index=False, encoding='utf-8-sig')
pd.DataFrame(star4).to_csv("파일명.csv", index=False, encoding='utf-8-sig')
pd.DataFrame(star3).to_csv("파일명.csv", index=False, encoding='utf-8-sig')
pd.DataFrame(star2).to_csv("파일명.csv", index=False, encoding='utf-8-sig')
pd.DataFrame(star1).to_csv("파일명.csv", index=False, encoding='utf-8-sig')

# 전체 저장
all_reviews = star5 + star4 + star3 + star2 + star1
df_all = pd.DataFrame(all_reviews)
df_all.to_csv("파일명.csv", index=False, encoding='utf-8-sig')